In [ ]:
from bs4 import BeautifulSoup
import requests

def votos2int(nvotos):
    """Convierte a entero una cadena que tiene un número que usa el "." como separador de miles
    
    Args:
        nvotos: cadena que representa un número de votos
        
    Returns:
        int: Número de votos representado por la cadena
    """
    
    return int(nvotos.replace(".",""))


def circunscripcion(nombre, url):
    """Extrae la información relativa a una circunscripción

    Para una circunscripción genera un diccionario con todos los datos extraidos de la circunscripcións

    Args:
        nombre (str): Nombre de la circunscripción para procesar.
        url (str): URL con los datos de la circusncripción.

    Returns:
        dict: Diccionario con los siguientes datos:
            circunscripcion: Nombre de la circunscripción
            url: URL de la web de la que se han extraido los datos
            escanyos: número de escaños que se reparten en la circunscripción
            votosTotales: Número total de votos contados
            abstencion: Número de abstenciones
            votosNulos: Número de votos nulos
            votosEnBlanco: Número de votos en blanco
            resultados: Lista con la distribución de los votos. Cada elemento de la lista es un diccionario con:
                partido: Nombre del partido político
                votos: número de votos obtenidos
                diputados: número de diputados conseguido
     Raises:
        NameError: La petición web a la URL ha provocado un error. Devuelve el código de error.
    """
    
    # Realizamos la petición a la web
    req = requests.get(url)

    # Comprobamos que la petición nos devuelve un Status Code = 200
    statusCode = req.status_code
    if statusCode == 200:
        # Pasamos el contenido HTML de la web a un objeto BeautifulSoup()
        html = BeautifulSoup(req.text, 'lxml')

        # Extracción del resumen de votos
        
        # Guardamos las filas de la tabla
        tablaResumenVotos = html.select("#TVRESUMEN tbody > tr")
               
        # Guardamos los datos de cada fila
        totalVotos = tablaResumenVotos[0].find('td').text;
        abstencion = tablaResumenVotos[1].find('td').text;
        nulos = tablaResumenVotos[2].find('td').text;
        blanco = tablaResumenVotos[3].find('td').text;
        
        
        # Extracción de la información por partidos
        partidos = html.select("tbody th.siglas15")
        votos= html.select("td.vots.s15")
        diputados= html.select("td.dip.s15")
        encanyos = html.select("#grafact span.numelegir")
        partidoVoto = []
        for i,p in enumerate(partidos):
            if len(p.text)>0:
                partidoVoto.append({'partido': p.text,
                                    'votos': votos2int(votos[i].text),
                                    'diputados': int(diputados[i].text or 0)
                                   })
        return {
            "circunscripcion": nombre,
            "url": url,
            "votosTotales": votos2int(totalVotos),
            "abstencion": votos2int(abstencion),
            "votosNulos": votos2int(nulos),
            "votosEnBlanco": votos2int(blanco),
            "escanyos": encanyos[0].text,
            "resultados": partidoVoto
        }
    else:
        raise NameError("Error Url request: " + statusCode)
        
# Test
circunscripcion("Madrid","http://resultados2016.infoelecciones.es/99CO/DCO12289CI.htm")

In [ ]:
from bs4 import BeautifulSoup
import requests

urlInicial = "http://resultados2016.infoelecciones.es/99CO/DCO99999TO.htm?lang=es"
urlBase = "http://resultados2016.infoelecciones.es/99CO/"

def recorrecircunscripciones (urlOrigen, urlBase):
    """Busca todas las circunscripciones y agrega los resultados obtenidos por cada circunscripción.
    
    Ignora los distritos: por ejemplo, Las Palmas se compone de Gran Canaria, Lanzarote y Tenerife.
    Estos resultados son ignorados y solo se guardan los de la circunscripción
    que las aglutina (en este caso, Las Palmas)
    
    Args:
        urlOrigen: URL desde la que se va a partir la búsqueda. Contiene un menú del que se extraerán las circunscripciones
        urlBase: URL a partir de la cual se componen las rutas a la información de las circunscripciones
    
    Returns:
        list: Una lista con la información agregada de cada una de las circunscripciones
    Raises:
        NameError: La petición web a la URL ha provocado un error. Devuelve el código de error.
    """

    # Realizamos la petición a la web
    req = requests.get(urlOrigen)

    # Comprobamos que la petición nos devuelve un Status Code = 200
    statusCode = req.status_code
    if statusCode == 200:
        # Pasamos el contenido HTML de la web a un objeto BeautifulSoup()
        html = BeautifulSoup(req.text, 'lxml')
        listaMunicipios= html.select("#b10_2 li.lici > a")
        resultadosTotales =[]
        for m in listaMunicipios:
            resultadosTotales.append(circunscripcion(m.text, urlBase+m['href']))
    
        return resultadosTotales
    else:
        raise NameError("Error Url request: " + statusCode)
 
# Test
recorrecircunscripciones(urlInicial, urlBase)

In [ ]:
from pprint import pprint

# Creamos un archivo con la lista de resultados en un json.
with open('resultados.json', 'wt') as out:
    pprint(recorrecircunscripciones(urlInicial, urlBase), stream=out)